In [2]:
import pandas as pd
import pickle
import numpy as np

In [2]:
test_creative_seq = pd.read_csv('test_creative_seq.csv')

FileNotFoundError: [Errno 2] File test_creative_seq.csv does not exist: 'test_creative_seq.csv'

In [3]:
test_creative_seq.head()

,user_id,creative_id
0,3000001,"[351878, 665090, 103064, 593698, 1508864, 1711..."
1,3000002,"[152519, 151984, 12838, 176984, 72773, 64667, ..."
2,3000003,"[161840, 73137, 115761, 36634, 367084, 150407,..."
3,3000004,"[108656, 849706, 678427, 9870, 157180, 94025, ..."
4,3000005,"[123860, 183003, 40625, 26793, 71219, 259607, ..."


In [12]:
from ast import literal_eval

def hh(q, item_dict, length):
    q = literal_eval(q)
    if len(q) < length:
        return [item_dict[x] for x in q] + [0]*(length-len(q))
    else:
        return [item_dict[x] for x in q[-length:]]

In [6]:
fp = open("item_dict.txt", "rb+")
item_dict = pickle.load(fp)

In [9]:
print(len(item_dict))
print(item_dict[8])

3257690
1725815


In [13]:
test_creative_seq["creative_id"]=test_creative_seq["creative_id"].apply(hh, args = (item_dict,89,))

In [14]:
test_creative_seq.head()

,user_id,creative_id
0,3000001,"[682852, 66840, 6632, 39833, 29629, 26034, 499..."
1,3000002,"[16307, 283417, 1281, 9890, 447, 1277, 1414, 5..."
2,3000003,"[105697, 24040, 9562, 8816, 452070, 467673, 78..."
3,3000004,"[3727, 650376, 96551, 15307, 29, 80097, 75217,..."
4,3000005,"[43718, 458791, 384, 179624, 263779, 3525, 317..."


In [15]:
test_creative_seq['seq_len'] = test_creative_seq['creative_id'].apply(lambda x: len(x))

In [16]:
test_creative_seq.head()

,user_id,creative_id,seq_len
0,3000001,"[682852, 66840, 6632, 39833, 29629, 26034, 499...",89
1,3000002,"[16307, 283417, 1281, 9890, 447, 1277, 1414, 5...",89
2,3000003,"[105697, 24040, 9562, 8816, 452070, 467673, 78...",89
3,3000004,"[3727, 650376, 96551, 15307, 29, 80097, 75217,...",89
4,3000005,"[43718, 458791, 384, 179624, 263779, 3525, 317...",89


In [17]:
test_creative_seq.to_csv('/DATA/zhouyongyu/taac/predict_creative_seq.csv',index=0)

# 真正开始

In [1]:
import pandas as pd
import pickle
import numpy as np

In [2]:
fp = open("/DATA/zhouyongyu/taac/predict_4_input.txt", "rb+")
(predict_creative, predict_ad, predict_advertiser, predict_mask, predict_user) = pickle.load(fp)

In [3]:
import torch.nn as nn
import torch

def trans_to_device(variable):
    if torch.cuda.is_available():
        return variable.cuda()
#         return variable
    else:
        return variable

class BiLSTM(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """
 
    def __init__(self, output_size_age, output_size_gender, embedding_dim, hidden_dim, n_layers, bidirectional=True, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(BiLSTM, self).__init__()
 
        self.output_size_age = output_size_age
        self.output_size_gender = output_size_gender
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        self.bidirectional = bidirectional
        
        # embedding and LSTM layers
#         self.embedding = nn.Embedding(vocab_size, embedding_dim)
#         self.embedding = embedding
        self.lstm1 = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True,
                            bidirectional=bidirectional)
        
        self.dropout1 = nn.Dropout(0.5)
        self.dropout2 = nn.Dropout(0.5)
        
        # dropout layer
#         self.dropout1 = nn.Dropout(0.5)
        
        self.lstm2 = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True,
                            bidirectional=bidirectional)
        
        # dropout layer
#         self.dropout2 = nn.Dropout(0.5)
        
        self.lstm3 = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True,
                            bidirectional=bidirectional)
        
        # dropout layer
#         self.dropout3 = nn.Dropout(0.5)
        
        # linear and sigmoid layers
        if bidirectional:
            self.bn0 = nn.BatchNorm1d(hidden_dim*6, momentum=0.5)
            self.fc1 = nn.Linear(hidden_dim*6, 200)
            self.bn1 = nn.BatchNorm1d(hidden_dim*6, momentum=0.5)
#             self.fc2 = nn.Linear(200, 200)
            self.fc_age = nn.Linear(200, output_size_age)
            self.bn_age = nn.BatchNorm1d(output_size_age, momentum=0.5)
#             self.fc_gender = nn.Linear(hidden_dim*2, output_size_gender)
        else:
            self.bn0 = nn.BatchNorm1d(hidden_dim*3, momentum=0.5)
            self.fc1 = nn.Linear(hidden_dim*3, 200)
            self.bn1 = nn.BatchNorm1d(200, momentum=0.5)
#             self.fc2 = nn.Linear(200, 200)
            self.fc_age = nn.Linear(200, output_size_age)
            self.bn_age = nn.BatchNorm1d(output_size_age, momentum=0.5)
#             self.fc_gender = nn.Linear(hidden_dim, output_size_gender)
          
        self.sig = nn.Sigmoid()
#         print("==before")
        self.reset_parameters()
#         print("==after")
#         print("==emb")
        self.creative_embedding = creative_embedding
        self.ad_embedding = ad_embedding
        self.advertiser_embedding = advertiser_embedding
#         print(self.embedding)
        
 
    def forward(self, creative, ad, advertiser, mask):
        creative_in = self.creative_embedding(creative)
        ad_in = self.ad_embedding(ad)
        advertiser_in = self.advertiser_embedding(advertiser)
        
        # ==========pack_pad===========
        
        lengths = torch.sum(mask, dim=1).int().cpu().numpy().tolist()
        
        lengths = torch.tensor(lengths)
#         print(lengths)
        _, idx_sort = torch.sort(lengths, dim=0, descending=True)
        lengths = list(lengths[idx_sort])

        # lstm1
        creative_in = creative_in.index_select(0, trans_to_device(idx_sort))  # 按下标取元素 
        
#         pack = torch.nn.utils.rnn.pack_padded_sequence(hidden_in, lengths, batch_first=True, enforce_sorted=False)
        pack_creative = torch.nn.utils.rnn.pack_padded_sequence(creative_in, lengths, batch_first=True)
        number = 1
        if self.bidirectional:
            number = 2
        h0 = trans_to_device(torch.randn(self.n_layers*number, creative_in.shape[0], self.hidden_dim))
        c0 = trans_to_device(torch.randn(self.n_layers*number, creative_in.shape[0], self.hidden_dim))
        out_creative, _ = self.lstm1(pack_creative, (h0, c0))
        
        output_padded_creative, _ = torch.nn.utils.rnn.pad_packed_sequence(out_creative, batch_first=True)
        
        #还原tensor
        _, idx_unsort = torch.sort(idx_sort)
        output_padded_creative = output_padded_creative.index_select(0, trans_to_device(idx_unsort))
        

        final_out1 = torch.max(output_padded_creative, 1)[0]
        
         # lstm2
        ad_in = ad_in.index_select(0, trans_to_device(idx_sort))  # 按下标取元素 
        
#         pack = torch.nn.utils.rnn.pack_padded_sequence(hidden_in, lengths, batch_first=True, enforce_sorted=False)
        pack_ad = torch.nn.utils.rnn.pack_padded_sequence(ad_in, lengths, batch_first=True)
        number = 1
        if self.bidirectional:
            number = 2
        h0 = trans_to_device(torch.randn(self.n_layers*number, ad_in.shape[0], self.hidden_dim))
        c0 = trans_to_device(torch.randn(self.n_layers*number, ad_in.shape[0], self.hidden_dim))
        out_ad, _ = self.lstm2(pack_ad, (h0, c0))
        
        output_padded_ad, _ = torch.nn.utils.rnn.pad_packed_sequence(out_ad, batch_first=True)
        
        #还原tensor
        _, idx_unsort = torch.sort(idx_sort)
        output_padded_ad = output_padded_ad.index_select(0, trans_to_device(idx_unsort))
        

        final_out2 = torch.max(output_padded_ad, 1)[0]
        
         # lstm3
        advertiser_in = advertiser_in.index_select(0, trans_to_device(idx_sort))  # 按下标取元素 
        
#         pack = torch.nn.utils.rnn.pack_padded_sequence(hidden_in, lengths, batch_first=True, enforce_sorted=False)
        pack_advertiser = torch.nn.utils.rnn.pack_padded_sequence(advertiser_in, lengths, batch_first=True)
        number = 1
        if self.bidirectional:
            number = 2
        h0 = trans_to_device(torch.randn(self.n_layers*number, advertiser_in.shape[0], self.hidden_dim))
        c0 = trans_to_device(torch.randn(self.n_layers*number, advertiser_in.shape[0], self.hidden_dim))
        out_advertiser, _ = self.lstm3(pack_advertiser, (h0, c0))
        
        output_padded_advertiser, _ = torch.nn.utils.rnn.pad_packed_sequence(out_advertiser, batch_first=True)
        
        #还原tensor
        _, idx_unsort = torch.sort(idx_sort)
        output_padded_advertiser = output_padded_advertiser.index_select(0, trans_to_device(idx_unsort))
        

        final_out3 = torch.max(output_padded_advertiser, 1)[0]
        
       
        # dropout and fully-connected layer
#         out1 = self.dropout3(final_out1)
#         out2 = self.dropout3(final_out2)
#         out3 = self.dropout3(final_out3)
#         out = final_out

#         out1 = self.dropout(final_out1)

        out = torch.cat([final_out1, final_out2, final_out3], 1)
    
        final_out1 = self.dropout1(out)
        final_out1 = self.bn0(final_out1)
        
        final_out1 = self.fc1(final_out1)
        final_out1 = self.bn1(final_out1)
        final_out1 = torch.relu(final_out1)
        final_out1 = self.dropout2(final_out1)

        final_out1 = self.fc_age(final_out1)
        final_out1 = self.bn_age(final_out1)
        final_out1 = torch.tanh(final_out1)

        return final_out1
#         return torch.tanh(out_age)

    def reset_parameters(self):
        # print("调用")
        import math
        stdv = 1.0 / math.sqrt(self.hidden_dim)
        for weight in self.parameters():
            # print("===stdv===")
            # print(stdv)
#             print(weight)
            weight.data.uniform_(-stdv, stdv)

In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
model = torch.load('/DATA/zhouyongyu/model_save/w2v32_age_model/2_copy_age_initmodel_4input_epoch_19.pkl_0.45904')
model = trans_to_device(model)

In [6]:
def generate_batch(creative, ad, advertiser, mask, user, batch_size):
    # 只是分成多个slices
    # slices中储存的为多个索引list（将索引切分为各个batch），如[[0, 1, 2], [3, 4, 5], ...]
#     creative = creative[:2000]
#     ad = ad[:2000]
#     advertiser = advertiser[:2000]
#     mask = mask[:2000]
#     age = age[:2000]
#     gender = gender[:2000]
    length = len(creative)
    n_batch = int(length / batch_size)
    if length % batch_size != 0:
        n_batch += 1
    # 等分成n_batch份, slices:n_batch*batch_size
    # 关于np.split()  https://www.jianshu.com/p/d020afd053bc
    slices = np.split(np.arange(n_batch * batch_size), n_batch)
    # 调整最后一份的length
    slices[-1] = slices[-1][:(length - batch_size * (n_batch - 1))]
    # slices中储存的为多个索引list（将索引切分为各个batch），如[[0, 1, 2], [3, 4, 5], ...]
    # slices是一个二维List，存放了一个batch里面的多个session
    return slices, np.array(creative), np.array(ad), np.array(advertiser), np.array(mask), np.array(user)

def get_slice(creative, ad, advertiser, mask, user, batch_index):
    # slice的形式 [[0, 1, 2], [3, 4, 5], ...]
    # 每一个slice就是一个batch，遍历slices得到的每一个batch_index就是一个数组，含有该batch内那几条数据的索引
    
    return creative[batch_index], ad[batch_index], advertiser[batch_index], mask[batch_index], user[batch_index]

In [7]:
from tqdm import tqdm

out_df = pd.DataFrame(columns=['user_id', 'predicted_age'])

melo_batch_size = 1024

slices, creative, ad, advertiser, mask, user = \
            generate_batch(predict_creative, predict_ad, predict_advertiser, predict_mask, predict_user, batch_size=melo_batch_size)

for slice in tqdm(slices):
    creative_melo, ad_melo, advertiser_melo, mask_melo, user_melo = \
                get_slice(creative, ad, advertiser, mask, user, batch_index=slice)
    
    creative_melo = trans_to_device(torch.Tensor(creative_melo).long())
    ad_melo = trans_to_device(torch.Tensor(ad_melo).long())
    advertiser_melo = trans_to_device(torch.Tensor(advertiser_melo).long())
    mask_melo = trans_to_device(torch.Tensor(mask_melo).float())
    
    
    out_age = model(creative_melo, ad_melo, advertiser_melo, mask_melo)
            
    out_age = torch.nn.functional.log_softmax(out_age, dim=1)
#     out_gender = torch.nn.functional.log_softmax(out_gender, dim=1)

    pred_age = out_age.argmax(1) + 1
#     pred_gender = out_gender.argmax(1) + 1
    
    c={"user_id": user_melo.tolist(),"predicted_age": pred_age.cpu().numpy().tolist()}
    insertRow = pd.DataFrame(c,columns = ['user_id','predicted_age'])
    
    out_df = out_df.append(insertRow,ignore_index=True)
    
#     print(len(out_df))

# for index, row in tqdm(test_creative_seq.iterrows()):
# #     if index==100:
# #         break
    
#     user_id = row['user_id']
#     creative_seq = row['creative_id']
    
#     input = [creative_seq]
#     mask = [[1 if item != 0 else 0 for item in creative_seq]]
#     input_melo = trans_to_device(torch.Tensor(input).long())
#     mask_melo = trans_to_device(torch.Tensor(mask).float())
    
#     out_age, out_gender = model(input_melo, mask_melo)
            
#     out_age = torch.nn.functional.log_softmax(out_age, dim=1)
#     out_gender = torch.nn.functional.log_softmax(out_gender, dim=1)

#     pred_age = out_age.argmax(1)
#     pred_gender = out_gender.argmax(1)
    
# #     print(pred_age)
# #     print(pred_age.item() + 1)
    
#     age = pred_age.item() + 1
#     gender = pred_gender.item() + 1
    
#     insertRow = pd.DataFrame([[user_id, age, gender]],columns = ['user_id','predicted_age','predicted_gender'])

#     out_df = out_df.append(insertRow,ignore_index=True)

out_df.to_csv('/DATA/zhouyongyu/taac/age-sep-519.csv',index=None)

100%|██████████| 977/977 [03:06<00:00,  5.23it/s]


In [8]:
out_df.head()

,user_id,predicted_age
0,3000001,3
1,3000002,7
2,3000003,3
3,3000004,3
4,3000005,3


In [11]:
another_df = pd.read_csv('/DATA/zhouyongyu/taac/gender-sep-518.csv')

In [12]:
another_df.head()

,user_id,predicted_gender
0,3000001,1
1,3000002,2
2,3000003,2
3,3000004,1
4,3000005,1


In [13]:
merge_submit_df = pd.merge(out_df, another_df, how='left', on=['user_id'])

In [14]:
merge_submit_df.head()

,user_id,predicted_age,predicted_gender
0,3000001,3,1
1,3000002,7,2
2,3000003,3,2
3,3000004,3,1
4,3000005,3,1


In [15]:
merge_submit_df.to_csv('/DATA/zhouyongyu/taac/submit_age_gender_sep-522.csv',index=None)